# 1 - LangServe

<img src="https://raw.githubusercontent.com/Hack-io-AI/ai_images/main/langchain.jpeg" style="width:400px;"/>

<h1>Tabla de Contenidos<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1---¿Qué-es-LangServe?" data-toc-modified-id="1---¿Qué-es-LangServe?-1">1 - ¿Qué es LangServe?</a></span></li><li><span><a href="#2---Servidor" data-toc-modified-id="2---Servidor-2">2 - Servidor</a></span></li><li><span><a href="#3---Cliente" data-toc-modified-id="3---Cliente-3">3 - Cliente</a></span></li></ul></div>

## 1 - ¿Qué es LangServe?

LangServe ayuda a los desarrolladores a desplegar runnables y cadenas de LangChain como una API REST. Esta biblioteca está integrada con FastAPI y utiliza pydantic para la validación de datos. Además, proporciona un cliente que se puede utilizar para llamar a runnables desplegados en un servidor, y también está disponible un cliente en JavaScript en LangChainJS. Para usar esta librería debemos ejecutar el siguiente comando:

```bash
pip install "langserve[all]"
```

<br>

**Características**:

+ Los esquemas de entrada y salida se infieren automáticamente del objeto LangChain y se aplican en cada llamada a la API, con mensajes de error detallados.

+ Está disponible una página de documentación de API con JSONSchema y Swagger.

+ Endpoints eficientes /invoke, /batch y /stream con soporte para muchas solicitudes concurrentes en un solo servidor.

+ Endpoint /stream_log para transmitir todos o algunos de los pasos intermedios de la cadena o el agente.

+ Página de playground en /playground con salida en tiempo real y pasos intermedios.

+ Todo construido con bibliotecas de Python de código abierto probadas en batalla como FastAPI o asyncio.


## 2 - Servidor

Para crear un API con LangServe, primero tenemos que crear el código del servidor, el cual es necesario ejecutar en un archivo `.py` para realizar el despliegue. Vamos a crear una API con LangServe usando una cadena sencilla.

```python
# librerias

from dotenv import load_dotenv 
import os

from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

from fastapi import FastAPI
import uvicorn
from langserve import add_routes



# carga de variables de entorno
load_dotenv()


# api key openai, nombre que tiene por defecto en LangChain
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')



# creacion de la aplicacion de FastAPI
app = FastAPI(title='Servidor LangChain', 
              version='1.0',
              description='Simple API con LangChain')


# inicio del modelo
modelo = ChatOpenAI()


# creacion del prompt
plantilla = 'Eres un asistente personal que responde la siguiente pregunta de la mejor manera: {pregunta}'

prompt = ChatPromptTemplate.from_template(plantilla)


# se define la cadena
cadena = prompt | modelo

# se añade el endpoint a la aplicacion
add_routes(app, cadena, path='/cadena')


# ejecucion de la API
if __name__ == '__main__':
    uvicorn.run(app, host='localhost', port=8000)
  

```

## 3 - Cliente

Para usar la API como cliente, una vez levantado el servidor, podemos usar el playground desde nuestro navegador usando la siguiente url:

http://localhost:8000/cadena/playground/

<br>

También podemos realizar la llamada la servidor desde una terminal usando curl de la siguiente manera:

```bash
curl --location --request POST 'http://localhost:8000/cadena/invoke/' \
     --header 'Content-Type: application/json' \
     --data-raw '{"input": {"pregunta": "hola"}}'
```

<br>

Incluso podriamos hacer un request desde Python con el siguiente código:

In [1]:
import requests as req

endpoint = 'http://localhost:8000/cadena/invoke/'

pregunta = 'hola'

respuesta = req.post(endpoint, json={'input': {'pregunta': pregunta}})

respuesta.json()

{'output': {'content': '¡Hola! ¿En qué puedo ayudarte hoy?',
  'additional_kwargs': {'refusal': None},
  'response_metadata': {'token_usage': {'completion_tokens': 11,
    'prompt_tokens': 27,
    'total_tokens': 38,
    'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0},
    'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}},
   'model_name': 'gpt-3.5-turbo-0125',
   'system_fingerprint': None,
   'finish_reason': 'stop',
   'logprobs': None},
  'type': 'ai',
  'name': None,
  'id': 'run-07c3cd3b-01be-4472-bd81-73f5d3b9e3b4-0',
  'example': False,
  'tool_calls': [],
  'invalid_tool_calls': [],
  'usage_metadata': {'input_tokens': 27,
   'output_tokens': 11,
   'total_tokens': 38,
   'input_token_details': {'cache_read': 0},
   'output_token_details': {'reasoning': 0}}},
 'metadata': {'run_id': '6513a484-038a-4a83-bb06-08087489c8b2',
  'feedback_tokens': []}}

In [2]:
# respuesta formato string

respuesta.json()['output']['content']

'¡Hola! ¿En qué puedo ayudarte hoy?'